## Ultralytics YOLOv8 

<a target="_blank" href="https://colab.research.google.com/github/dyi-el/optimized-traffic-light-computer-vision/blob/main/YOLOv8.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

For training using Google Colab

### Setup
Install and check dependencies

In [ ]:
%pip install fiftyone
%pip install ultralytics
import ultralytics
ultralytics.checks()

Using FiftyOne https://docs.voxel51.com/user_guide/dataset_zoo/datasets.html#coco-2017, we can download the needed annotated data from COCO.

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

# Define the classes and maximum samples per class
classes = ["person", "bicycle", "motorcycle", "bus", "truck", "car"]

# Load COCO dataset with annotations filtered for classes
detection_dataset = foz.load_zoo_dataset(
    "coco-2017",
    splits=["train","validation"],
    label_types=["detections"],
    classes=classes,
    include_id=False,
    include_license=False,
    only_matching=True,
    seed=12,
    max_samples=8000, 
    progress=True
)

In [ ]:
# Export the filtered dataset to COCO format
export_dir = "/coco"

# The splits to export
splits = ["train", "validation"]

# Export the splits
for split in splits:
    split_view = detection_dataset.match_tags(split)
    split_view.export(
        export_dir=export_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field="ground_truth",
        split=split,
        classes=classes,
    )

### Train

In [ ]:
#@title Select YOLOv8 🚀 logger {run: 'auto'}
logger = 'Comet' #@param ['Comet', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir .

Move the dataset.yaml to the working directory.

In [ ]:
# Training mode
!yolo train model=yolov8n.pt data=dataset.yaml epochs=3 imgsz=640

### Validation

In [ ]:
# Validation mode
!yolo val model=yolov8n.pt data=dataset.yaml

### Export

In [ ]:
!yolo export model=yolov8n.pt format=torchscript